Transfer learning using cryptobert and roberta

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import pandas as pd
from scipy.special import softmax

/home/peter/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ElKulako/cryptobert")

model = AutoModelForSequenceClassification.from_pretrained("ElKulako/cryptobert")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

In [ ]:
model.type

In [ ]:
pwd

Importing a sample of some Bitcoin Tweet Data from Kaggle to begin analysing the model

In [3]:
data = pd.read_csv('raw_data/bitcoin_tweets1000000.csv', nrows = 1000)

# "\\wsl.localhost\Ubuntu\home\peter\code\giadapi\crypto\kaggle-tweets.zip"

In [4]:
data

,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,cleanText,Polarity Score,sentiment
0,0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False,blue ridge bank shares halted nyse bitcoin atm...,0.00,0
1,1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False,today thursday take friend leowandersleb btc w...,0.00,0
2,2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False,guys evening read article btc would like share,0.00,0
3,3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False,btc big chance billion price bitcoin btc crypto,0.00,0
4,4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False,network secured nodes today soon biggest bears...,-0.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Szymon Kostrzewa,NaN,NaN,2016-04-02 12:33:50,7.0,75,31,False,2021-02-10 17:27:33,🤝 Follow me on @betfury_io. Let's hunt for Bit...,NaN,Twitter Web App,False,follow betfury_io let hunt bitcoins together g...,0.30,1
996,996,Crypto Zombie,New York,🎥 YouTube: Crypto Zombie 🧟‍♂️ #Bitcoin and #bl...,2010-01-18 23:52:55,59416.0,1058,23385,False,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,['Bitcoin'],Twitter Web App,False,bitcoin falling tesla pump top btc worried dip...,0.50,1
997,997,Brian Harrington,"Anaheim, CA",#Bitcoin is actively replacing the dollar. Can...,2009-08-19 20:21:01,7687.0,2322,83966,False,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,NaN,Twitter Web App,False,within driving distance iowa city definitely c...,0.00,0
998,998,🔸️Bitcoin Pixel🔸️,NaN,I discovered crypto in 2019 on Youtube. Since ...,2015-07-08 13:44:42,1873.0,75,676,False,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,"['BTC', 'ETH', 'DOT']",Twitter for Android,False,question please finally sold avax amp cake bou...,0.00,0


In [5]:
data = data[['text', 'date']]

In [6]:
data['text'][0]

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement https://t.co/xaaZmaJKiV @MyBlueRidgeBank… https://t.co/sgBxMkP1SI'

In [11]:
# Preprocess text (username and link placeholders) - this is suggested from the example in 
# https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment
def preprocess_1(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

preprocess_1(data['text'][0])

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement http @user http'

In [12]:
# I have changed this to remove more information

def preprocess_2(text):
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

preprocess_2(data['text'][0])

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement   '

In [ ]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)